In [1]:
import pandas as pd
import defusedxml.ElementTree as ET
# import re

## Školy

In [2]:
tree = ET.parse('vrejcelk.xml')
root = tree.getroot()

In [3]:
skoly_data = []
zarizeni_info_data = []
zarizeni_misto_data = []
skola_obor_data = []

for ps in root.findall('.//PravniSubjekt'):
    # TAB "skoly" a "zarizeni_info"
    ico = ps.find('ICO').text
    nazev = ps.find('Reditelstvi').find('RedPlnyNazev').text
    for sz in ps.find('SkolyZarizeni').findall('SkolaZarizeni'):
        izo = sz.find('IZO').text
        typZarizeni = sz.find('SkolaPlnyNazev').text
        zarizeniDruhTyp = sz.find('SkolaDruhTyp').text
        zarizeniKapacita = sz.find('SkolaKapacita').text
        kapacitaJednotka = sz.find('SkolaKapacitaJednotka').text
        jazyk = sz.find('SkolaJazyk').text

        raw_skoly = {'IZO' : izo,  
                    'ZarizeniTyp': typZarizeni, 
                    'Kapacita': zarizeniKapacita,
                    'KapacitaJednotka': kapacitaJednotka,           
                    }
        raw_zarizeni_info = {
                    'ICO': ico, 
                    'ReditelstviNazev' : nazev, 
                    'ZarizeniDruhTyp' : zarizeniDruhTyp, 
                    'jazyk' : jazyk, 
                    'IZO' : izo,
                    }
        skoly_data.append(raw_skoly)
        zarizeni_info_data.append(raw_zarizeni_info)

        # TAB "zarizeni_misto"
        for smvc in sz.find('SkolaMistaVykonuCinnosti').findall('SkolaMistoVykonuCinnosti'):
            idMista = smvc.find('IDMista').text
            adresaX = smvc.find('MistoAdresa2').text
            adresaY = smvc.find('MistoAdresa3').text
            druh = smvc.find('MistoDruhTyp').text # u středních škol např. rozlišení gymnázií, u MŠ specializace
            # adresu možná nahradit přes (nutno poupravit kod) RUIAN API, nebo https://data.gov.cz/datov%C3%A1-sada?iri=https%3A%2F%2Fdata.gov.cz%2Fzdroj%2Fdatov%C3%A9-sady%2F00025712%2F59ddccd404a617f695e3a1ef8e65b81e 
                             
            raw_zarizeni_misto = {
                    'druh' : druh,
                    'IDMista': idMista,
                    'adresaX' : adresaX,
                    'adresaY' : adresaY,
                    'IZO' : izo,
                   }
            zarizeni_misto_data.append(raw_zarizeni_misto)
            
        # TAB "skola_obor"
        if sz.find('SkolaOboryVzdelani') is not None:
            for sov in sz.find('SkolaOboryVzdelani').findall('SkolaOborVzdelani'):
                oborNazev = sov.find('OborNazev').text
                delkaVzdelavani = sov.find('DelkaVzdelavani').text
                oborKapacita = sov.find('OborKapacita').text
                oborDobihajici = sov.find('OborDobihajici').text
                # print(oborNazev)
    
                raw_skola_obor = {
                    'OborNazev' : oborNazev,
                    'DelkaVzdelavani': delkaVzdelavani,
                    'OborKapacita' : oborKapacita,
                    'OborDobihajici' : oborDobihajici,
                    'IZO' : izo,
                   }
                skola_obor_data.append(raw_skola_obor)

### Školy data -> dataframe

In [4]:
skoly = pd.DataFrame(skoly_data)

zarizeni_info = pd.DataFrame(zarizeni_info_data)
zarizeni_info.insert(loc=0, column='ID', value=range(1, len(zarizeni_info_data) + 1))
    
zarizeni_misto = pd.DataFrame(zarizeni_misto_data)
zarizeni_misto.insert(loc=0, column='ID', value=range(1, len(zarizeni_misto) + 1))
    
skola_obor = pd.DataFrame(skola_obor_data)
skola_obor.insert(loc=0, column='ID', value=range(1, len(skola_obor_data) + 1))

# kontrola dat u "skoly" a "zarizeni_info" v pořádku - stejný počet záznamů

## MŠ + PSZŠSpeciální TAB

In [5]:
materske_skoly = skoly[skoly['ZarizeniTyp'].isin(["Mateřská škola", "Mateřská škola (lesní mateřská škola)", "Mateřská škola - Przedszkole", "Mateřská škola s p.j.v.", "Přípravný stupeň základní školy speciální"])]
materske_skoly = materske_skoly.astype({'Kapacita':'int'})
# materske_skoly.info()
# materske_skoly['Kapacita'].sum()

## DětskéSkupinky TAB

In [6]:
from web_scrap_DS import DS_tabulka

In [7]:
DS_tabulka

,Kraj,Kapacita
0,Hlavní město Praha,4439
1,Jihočeský,1305
2,Jihomoravský,4029
3,Karlovarský,256
4,Královéhradecký,1113
5,Liberecký,758
6,Moravskoslezský,2198
7,Olomoucký,1387
8,Pardubický,1044
9,Plzeňský,1053


## Porodnost 2018-2023 TAB 

In [8]:
from web_scrap_porodnost import porodnost_tabulka

In [9]:
porodnost_tab = porodnost_tabulka.astype({"PocetNarozenychDeti":'int'}) 
porodnost_tabCR = porodnost_tab[porodnost_tab['Kraj'] == 'Česká republika']
porodnost_tab_kraje = porodnost_tab[porodnost_tab['Kraj'] != 'Česká republika']
# porodnost_tab_kraje
# porodnost_tabCR[porodnost_tabCR['Rok'].isin([2021, 2020, 2019, 2018])]['PocetNarozenychDeti'].sum()
# porodnost_tab.info()